# Setup

In [2]:
from openai import OpenAI
client = OpenAI()
import os

delete_intermediate_files = True #Delete intermediate files after processing

text_to_speech_output_folder = 'text_to_speech_outputs' #Folder for the outputs
# Create the directory if it doesn't already exist
if not os.path.exists(text_to_speech_output_folder):
    os.makedirs(text_to_speech_output_folder)
    print(f"Directory '{text_to_speech_output_folder}' created successfully.")
else:
    print(f"Directory '{text_to_speech_output_folder}' already exists.")


Directory 'text_to_speech_outputs' already exists.


#### basic test

In [3]:
response = client.audio.speech.create(
    model="tts-1",
    voice="alloy",
    input="Hello world! This is a streaming test.",
)

response.stream_to_file( os.path.join(text_to_speech_output_folder, "hello_world.mp3"))

/var/folders/78/7l8366_n3pj6jn47gs4m1dkh0000gn/T/ipykernel_46811/2055507718.py:7: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file( os.path.join(text_to_speech_output_folder, "hello_world.mp3"))


## Can I make it read html files?

First getting the html file

In [4]:
import requests

# Define the URL to be fetched
url = "https://importai.substack.com/p/import-ai-380-distributed-13bn-parameter"
#set the name of the file to save the final mp3 to
description_for_mp3_name = 'import_ai_380'

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Get the HTML content of the page
    html_content = response.text
    print("HTML content saved to the variable 'html_content'.")

else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


HTML content saved to the variable 'html_content'.


In [8]:
html_content

'<!DOCTYPE html>\n<html lang="en">\n    <head>\n        <meta charset="utf-8" />\n        <meta name="norton-safeweb-site-verification" content="24usqpep0ejc5w6hod3dulxwciwp0djs6c6ufp96av3t4whuxovj72wfkdjxu82yacb7430qjm8adbd5ezlt4592dq4zrvadcn9j9n-0btgdzpiojfzno16-fnsnu7xd" />\n        \n        <link rel="preconnect" href="https://substackcdn.com" />\n        \n\n        \n\n        \n        <link rel="preload" as="style" href="https://substackcdn.com/bundle/theme/main.4e82ef53281a3e5e1261.css" />\n        \n        \n        \n        <link rel="preload" as="font" href="https://fonts.gstatic.com/s/spectral/v13/rnCr-xNNww_2s0amA9M5knjsS_ul.woff2" crossorigin />\n        \n\n        \n            \n                <link type="text/css" rel="stylesheet" href="https://substackcdn.com/bundle/assets/entry-dd50ef76.css" />\n            \n                <link type="text/css" rel="stylesheet" href="https://substackcdn.com/bundle/assets/index-2caf47c8.css" />\n            \n                <

The above is pretty nasty - it has a bunch of html formatting stuff going on, which I don't want to hear when it's reading. Using bs4 to remove the formatting

In [5]:
from bs4 import BeautifulSoup

# Parse the HTML
soup = BeautifulSoup(html_content, 'html.parser')

# Extract text and strip whitespace
content_text = soup.get_text(separator='\n', strip=True)

# Print the cleaned text
print(content_text)

Import AI 380: Distributed 1.3bn parameter LLM; math AI; and why reality is hard for Ai
Subscribe
Sign in
Share this post
Import AI 380: Distributed 1.3bn parameter LLM; math AI; and why reality is hard for Ai
importai.substack.com
Copy link
Facebook
Email
Note
Other
Import AI 380: Distributed 1.3bn parameter LLM; math AI; and why reality is hard for Ai
What is our responsibility to machines that may become moral patients?
Jack Clark
Jul 29, 2024
11
Share this post
Import AI 380: Distributed 1.3bn parameter LLM; math AI; and why reality is hard for Ai
importai.substack.com
Copy link
Facebook
Email
Note
Other
1
Share
Welcome to Import AI, a newsletter about AI research. Import AI runs on lattes, ramen, and feedback from readers. If you’d like to support this (and comment on posts!) please subscribe.
Subscribe
Cambridge researchers show how to use distributed training to make a 1.3bn parameter LLM:
…More evidence that distributed training works well for relatively small models…
Researche

The above has gotten rid of the html stuff, which is great, but it also has a bunch of text from links at the bottom and top etc that I wouldn't want to read. I think this is a good use case for GPT4, so setting up a version of it with instructions to help me get rid of the parts that I wouldn't be interested in reading.

I found I had to tell it explicitly to NOT summarise or paraphrase at all, but it seems to do a pretty sweet job, especially after reducing the temperature to increase predictability

In [6]:
text_for_reader_object = client.chat.completions.create(
  model="gpt-4o-mini",
  # model="gpt-3.5-turbo", # this ran in about half the time, but it cut out the last paragraph
  # model="gpt-4", #this took 4x as long to run as the gpt-4o-mini model
  messages=[
    {"role": "system", "content": "You are an assistant who will help me out with cleaning up some text, so the relevant parts can be read out in a text-to-speech convertor. I have taken this text from a HTML file and removed all the tags. However, it still may have a bunch of headings and links that were in the orignal, but not of interest to someone reading the web page. Can you please remove the unnecessary headings and links from the text, so that the reader will just read what I want to hear? Make sure you include all relevant content from the whole document though. I DO NOT want you to summarise the text or change any of the words, outside of removing the headings and links. Please wrap the text in your output at 120 characters width"},
    {"role": "user", "content": f"Can please remove the unnecessary headings and links from this text {content_text}"}
  ],
  temperature=0.2 # I found that setting the temperature lower made sure that nothing was cut out of the text
)

text_for_reader = text_for_reader_object.choices[0].message.content


In [7]:
print(text_for_reader)


What is our responsibility to machines that may become moral patients? 

Welcome to Import AI, a newsletter about AI research. Import AI runs on lattes, ramen, and feedback from readers. If you’d like to support this (and comment on posts!) please subscribe.

Cambridge researchers show how to use distributed training to make a 1.3bn parameter LLM: More evidence that distributed training works well for relatively small models. Researchers with the University of Cambridge and Flower Labs have shown that it's possible to use cheap, distributed training approaches to train LLMs at the billion-parameter scale, providing more clues that in the future, some AI models could be trained via collectives pooling their hardware akin to the filesharing communities that developed around BitTorrent.

What is distributed training and why should you care? Today, frontier AI systems are trained in large data centers that contain lots of computers which are densely networked together. This means that trai

This is now in the format I want. I had a quick try at converting this into mp3 from here, but I've run up against a character limit of 4096. So I'm going to need to split my input up into a list of strings, convert them into mp3 individually, then look to concatenate them.

Note that the cost of the conversion isn't that cheap (1.5 cents per 1000 characters (USD)), so printing out expected cost here for user's awareness

In [24]:
expected_cost_USD = len(text_for_reader)/1000*0.015  #cost 0.015 USD per 1000 characters

print(f"Expected cost of text-to-speech conversion: US${round(expected_cost_USD,2)}")

Expected cost of text-to-speech conversion: US$0.24


In [16]:
def split_string(long_string, max_length=4096):
    # Create an empty list to hold the substrings
    result = []
    
    # Calculate the total length of the input string
    total_length = len(long_string)
    
    # Loop through the string and split it into substrings of max_length
    for i in range(0, total_length, max_length):
        substring = long_string[i:i + max_length]
        result.append(substring)
    
    return result

# Split the text into parts of 4096 characters each
text_for_reader_split = split_string(text_for_reader)

# Print the result to check the output
for idx, part in enumerate(text_for_reader_split):
    print(f"Part {idx + 1}: Length = {len(part)}")


Part 1: Length = 4096
Part 2: Length = 4096
Part 3: Length = 4096
Part 4: Length = 3720


In [17]:
text_for_reader_split

['What is our responsibility to machines that may become moral patients? \n\nWelcome to Import AI, a newsletter about AI research. Import AI runs on lattes, ramen, and feedback from readers. If you’d like to support this (and comment on posts!) please subscribe.\n\nCambridge researchers show how to use distributed training to make a 1.3bn parameter LLM: more evidence that distributed training works well for relatively small models. Researchers with the University of Cambridge and Flower Labs have shown that it\'s possible to use cheap, distributed training approaches to train LLMs at the billion-parameter scale, providing more clues that in the future, some AI models could be trained via collectives pooling their hardware akin to the filesharing communities that developed around BitTorrent.\n\nWhat is distributed training and why should you care? Today, frontier AI systems are trained in large data centers that contain lots of computers which are densely networked together. This means 

Time to convert to an mp3 file! I'm enjoying the 'alloy' voice the most

In [27]:
from tqdm import tqdm
split_mp3_paths = []
for i in tqdm(range(len(text_for_reader_split))):
    print(f"converting part {i+1} of {len(text_for_reader_split)}")
    response = client.audio.speech.create(
        model="tts-1",
        voice="alloy",
        input=text_for_reader_split[i]
    )
    
    split_mp3_name= f"{description_for_mp3_name}_{i+1}.mp3"
    split_mp3_path = os.path.join(text_to_speech_output_folder, split_mp3_name)
    response.stream_to_file( split_mp3_path )
    split_mp3_paths.append(split_mp3_path)
print("finished saving all parts")
print(f"saved mp3s: {split_mp3_paths}")

  0%|          | 0/4 [00:00<?, ?it/s]

saving part 1 of 4


/var/folders/78/7l8366_n3pj6jn47gs4m1dkh0000gn/T/ipykernel_35887/2849658267.py:13: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file( split_mp3_path )
 25%|██▌       | 1/4 [00:36<01:48, 36.18s/it]

saving part 2 of 4


 50%|█████     | 2/4 [01:14<01:14, 37.41s/it]

saving part 3 of 4


 75%|███████▌  | 3/4 [01:54<00:38, 38.78s/it]

saving part 4 of 4


100%|██████████| 4/4 [02:15<00:00, 33.90s/it]

finished saving all parts
saved mp3s: ['text_to_speech_outputs/import_ai_380_1.mp3', 'text_to_speech_outputs/import_ai_380_2.mp3', 'text_to_speech_outputs/import_ai_380_3.mp3', 'text_to_speech_outputs/import_ai_380_4.mp3']


Next, want to merge these mp3s together back into one. To do this you need ffmpeg, which I installed with conda using `conda install -c conda-forge ffmpeg`. However, it can also be downloaded from ffmpeg.org if you're not using conda. 

In [28]:
from pydub import AudioSegment

# Initialize an empty AudioSegment
combined = AudioSegment.empty()

# Loop through the list of MP3 files and append them to the combined AudioSegment
for mp3_file in split_mp3_paths:
    audio = AudioSegment.from_mp3(mp3_file)
    combined += audio

merged_mp3_path = os.path.join(text_to_speech_output_folder, f"{description_for_mp3_name}.mp3")
# Export the combined AudioSegment as a new MP3 file
combined.export( merged_mp3_path , format="mp3")

print("MP3 files have been merged successfully!")


MP3 files have been merged successfully!


In [31]:
# clearing these files to save space
if delete_intermediate_files:
    # Delete the intermediate MP3 files
    for mp3_file in split_mp3_paths:
        os.remove(mp3_file)
    print("Intermediate MP3 files have been deleted successfully!")

Intermediate MP3 files have been deleted successfully!
